In [ ]:
# 🔥 Firecracker Code Interpreter — Persistent Session Architecture

```
 AI Agents / Users                                    Host Process
 ─────────────────                            ─────────────────────────────
  session "alice"  ──┐                        ┌────────────────────────────┐
  session "bob"    ──┼──► CodeInterpreterTool │      SessionManager        │
  session "carol"  ──┘    (BaseTool subclass) │                            │
                           │ session_id attr  │ "alice" → SessionInfo(VM₁) │
                           ▼                  │ "bob"   → SessionInfo(VM₂) │
                    session.execute()         │ "carol" → SessionInfo(VM₃) │
                           │                  │                            │
                           │ (new session)    │  idle timeout: 30 min      │
                           ▼                  │  background cleanup task   │
                    ┌─────────────┐           └──────────────┬─────────────┘
                    │  WarmVMPool │  ◄── acquire              │ destroy (expired)
                    │ [VM][VM][VM]│      (pre-booted)         ▼
                    └─────────────┘                  vm.destroy() +
                                                     pool.replenish()

 Per-Session VM Lifecycle
 ────────────────────────────────────────────────────────────────────
  Host                                     Firecracker microVM
  ────                                     ──────────────────────────────
  SessionManager.execute(session, req)     Guest Agent v2 (persistent loop)
        │                                       │
        │── vsock CONNECT ─────────────────────►│
        │── JSON {type, code, timeout} ────────►│  exec(code, self.globals)
        │                                       │       ↑
        │◄─ JSON {success, output, error} ──────│  state persists forever
        │── vsock CLOSE                         │  (until reset or shutdown)
        │                                       │
        │  [next call, same session]            │  self.globals still has
        │── vsock CONNECT ─────────────────────►│  all your variables! 🎯
        │── JSON {type="bash", cmd} ───────────►│
        │◄─ JSON {output, exit_code} ───────────│

 Request Types
 ─────────────────────────────────────────────────────────────────────
  python     →  exec(code, session_globals)   # state persists between calls
  bash       →  /bin/bash -c "cmd"            # full shell access
  write_file →  open(path, "w").write(...)    # persist files in VM
  read_file  →  open(path).read()
  list_files →  os.scandir(path)
  install    →  pip3 install <packages>
  get_state  →  list all defined variables
  reset      →  clear globals, delete /tmp/exec_*.py
  ping       →  liveness check
  shutdown   →  graceful VM poweroff
```

**Key design:** One VM per `session_id` · State persists across calls · 1 vCPU · 256 MB RAM · KVM-accelerated · vsock IPC

=== Checking Prerequisites ===
✅ All prerequisites met!

=== Testing Code Interpreter ===
Copying rootfs (this may take a moment)...
Starting Firecracker VM...
Configuring VM...
Booting VM...
VM is running. To make this fully functional, you need:
1. A custom rootfs with a Python runtime
2. An init script that reads code from serial/vsock
3. A way to capture and return output

=== Result ===
{
  "success": true,
  "output": "VM executed successfully (stub implementation)",
  "note": "This is a simplified version showing VM lifecycle. Full implementation needs custom rootfs with code execution agent.",
  "vm_stdout": "final.target\u001b - Late Shutdown Services.\r\r\n[\u001b  OK  \u001b[0m] Finished \u001bsystemd-reboot.service\u001b - System Reboot.\r\r\n[\u001b  OK  \u001b[0m] Reached target \u001breboot.target\u001b - System Reboot.\r\r\n[    3.185044] reboot: Restarting system\r\n2026-02-22T17:37:15.634743087 [anonymous-instance:main] Vmm is stopping.\n2026-02-22T17:37:15.635226587 

## 1. Prerequisites Check

In [3]:
import sys, os, subprocess
sys.path.insert(0, "/home/administrator/Desktop/datascience/github/agent-framework/src")

# Apply nest_asyncio so asyncio.run() works inside Jupyter's event loop
import nest_asyncio; nest_asyncio.apply()
import asyncio

checks = {
    "Firecracker binary":  os.path.exists("/usr/local/bin/firecracker"),
    "KVM device":          os.path.exists("/dev/kvm"),
    "vsock device":        os.path.exists("/dev/vhost-vsock"),
    "Kernel image":        os.path.exists("/home/administrator/vmlinux-6.1.155"),
    "Rootfs (v2 agent)":   os.path.exists("/tmp/ci-rootfs-update.ext4"),
}

# Verify the rootfs has the v2 agent
if checks["Rootfs (v2 agent)"]:
    out = subprocess.run(
        ["debugfs", "/tmp/ci-rootfs-update.ext4", "-R", "cat /opt/agent/agent.py"],
        capture_output=True, text=True
    )
    has_v2 = "GuestAgent" in out.stdout and "exec_python" in out.stdout
    checks["Agent v2 in rootfs"] = has_v2

all_ok = all(checks.values())
for name, ok in checks.items():
    print(f"  {'✅' if ok else '❌'}  {name}")
print(f"\n{'🚀 All systems go!' if all_ok else '⚠️  Fix issues above before continuing'}")


  ✅  Firecracker binary
  ✅  KVM device
  ✅  vsock device
  ✅  Kernel image
  ✅  Rootfs (v2 agent)
  ✅  Agent v2 in rootfs

🚀 All systems go!


## 2. Session Manager — Boot & First Execution

Start the `SessionManager` (which internally boots the warm VM pool).  
The first call to a new `session_id` pulls a pre-booted VM from the pool.

In [4]:
import time, json, logging
logging.basicConfig(level=logging.WARNING)

from agent_framework.tools.code_interpreter import (
    SessionManager, CodeInterpreterConfig
)

config = CodeInterpreterConfig(
    pool_size=2,
    session_timeout=1800,   # 30 min idle → destroy VM
)

sm = SessionManager(config=config)

t0 = time.monotonic()
await sm.start()
print(f"✅ SessionManager started in {time.monotonic()-t0:.2f}s  ({config.pool_size} warm VMs ready)")

# First call — pulls a pre-booted VM from warm pool, assigns to session "alice"
t1 = time.monotonic()
r = await sm.execute("alice", {"type": "python", "code": "print('Hello from VM!')  \nimport numpy, pandas; print(f'numpy {numpy.__version__}  pandas {pandas.__version__}')"})
elapsed = time.monotonic() - t1

print(f"\n── Session 'alice' first call  ({elapsed*1000:.0f} ms) ──")
print(r["output"])
print("\nActive sessions:", sm.list_sessions())


✅ SessionManager started in 1.20s  (2 warm VMs ready)

── Session 'alice' first call  (1700 ms) ──
Hello from VM!
numpy 2.4.2  pandas 3.0.1


Active sessions: [{'session_id': 'alice', 'vm_id': '96e546776ba1', 'vm_state': 'READY', 'exec_count': 1, 'age_seconds': 2, 'idle_seconds': 0}]


## 3. Python State Persistence

Variables defined in one call are visible in the next — **same VM, same Python namespace**.  
This is exactly how Claude / OpenAI Code Interpreter works per conversation.

In [5]:
SID = "alice"  # same session as above

# Call 1 — define variables + build a DataFrame
r1 = await sm.execute(SID, {"type": "python", "code": """
import numpy as np
import pandas as pd

x = 42
data = np.random.seed(7)
df = pd.DataFrame({"A": np.random.randn(5), "B": np.arange(5)})
message = "State is alive!"
print(f"x={x}  df.shape={df.shape}  message={message!r}")
"""})
print("── Call 1 (define variables) ──")
print(r1["output"])

# Call 2 — use variables from call 1 WITHOUT re-defining them
r2 = await sm.execute(SID, {"type": "python", "code": """
# x, df, message were defined in the PREVIOUS call
print(f"x is still: {x}")
print(f"df still has {len(df)} rows")
print(df.to_string())
print(f"\\nMessage: {message}")
"""})
print("\n── Call 2 (variables from call 1 still live) ──")
print(r2["output"])

# Inspect session state
state = await sm.execute(SID, {"type": "get_state"})
print(f"\n── Session state after {state['exec_count']} executions ──")
for var, info in state["variables"].items():
    print(f"  {var}: {info['type']}  = {info['repr']}")


── Call 1 (define variables) ──
x=42  df.shape=(5, 2)  message='State is alive!'


── Call 2 (variables from call 1 still live) ──
x is still: 42
df still has 5 rows
          A  B
0  1.690526  0
1 -0.465937  1
2  0.032820  2
3  0.407516  3
4 -0.788923  4

Message: State is alive!


── Session state after 3 executions ──
  numpy: module  = module<function shape at 0x7ffb3932cb80>
  pandas: module  = <module 'pandas' from '/usr/local/lib/python3.12/dist-packages/pandas/__init__.py'>
  np: module  = module<function shape at 0x7ffb3932cb80>
  pd: module  = <module 'pandas' from '/usr/local/lib/python3.12/dist-packages/pandas/__init__.py'>
  x: int  = 42
  data: NoneType  = None
  df: DataFrame  = DataFrame(5, 2)
  message: str  = str[15]


## 4. Full VM Capabilities — Bash & File I/O

The guest agent exposes the full Firecracker VM: bash commands, filesystem writes, etc.  
This is the **"cat > executor.py && python3 executor.py"** power level.

In [6]:
SID = "alice"

# ── Bash execution ────────────────────────────────────────────────────────────
bash_r = await sm.execute(SID, {"type": "bash", "cmd": "uname -a && cat /etc/os-release | grep PRETTY && python3 --version && pip3 list 2>/dev/null | head -10"})
print("── Bash: system info ──")
print(bash_r["output"])

# ── Write a Python script to the VM then run it via bash ─────────────────────
SCRIPT = """\
import sys
print(f"Python {sys.version}")
xs = list(range(1, 11))
print(f"Sum 1-10 = {sum(xs)}")
print("Script ran via: cat > /tmp/runner.py && python3 /tmp/runner.py")
"""

wf = await sm.execute(SID, {"type": "write_file", "path": "/tmp/runner.py", "content": SCRIPT})
print(f"\n── write_file → {wf['path']}  ({wf['bytes_written']} bytes) ──")

run_r = await sm.execute(SID, {"type": "bash", "cmd": "python3 /tmp/runner.py"})
print("\n── python3 /tmp/runner.py (via bash) ──")
print(run_r["output"])

# ── List files in /tmp ────────────────────────────────────────────────────────
ls = await sm.execute(SID, {"type": "list_files", "path": "/tmp"})
print("\n── /tmp contents ──")
for e in ls["entries"][:10]:
    icon = "📁" if e["is_dir"] else "📄"
    print(f"  {icon} {e['name']:30s} {e['size']:>8} bytes")


── Bash: system info ──
Linux ubuntu-fc-uvm 6.1.155 #1 SMP PREEMPT_DYNAMIC Tue Nov 18 09:27:27 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
PRETTY_NAME="Ubuntu 24.04.3 LTS"
Python 3.12.3
Package         Version
--------------- ----------
boto3           1.34.46
botocore        1.34.46
certifi         2023.11.17
chardet         5.2.0
contourpy       1.3.3
cycler          0.12.1
fonttools       4.61.1
idna            3.6


── write_file → /tmp/runner.py  (169 bytes) ──

── python3 /tmp/runner.py (via bash) ──
Python 3.12.3 (main, Jan 22 2026, 20:57:42) [GCC 13.3.0]
Sum 1-10 = 55
Script ran via: cat > /tmp/runner.py && python3 /tmp/runner.py


── /tmp contents ──
  📁 .ICE-unix                            40 bytes
  📁 .X11-unix                            40 bytes
  📁 .XIM-unix                            40 bytes
  📁 .font-unix                           40 bytes
  📄 exec_0001.py                        121 bytes
  📄 exec_0002.py                        232 bytes
  📄 exec_0003.py                    

## 5. Multi-Session Isolation

`"alice"` and `"bob"` each get their own VM.  
Variables in one session are **completely invisible** to the other.

In [7]:
import asyncio

# Run alice and bob concurrently — each gets their own VM
async def run_sessions():
    # Alice defines her own secret variable
    await sm.execute("alice", {"type": "python", "code": "secret = 'alice_data'; counter = 100"})

    # Bob defines a different variable with the same name
    await sm.execute("bob", {"type": "python", "code": "secret = 'bob_data'; counter = 999"})

    # Alice's counter is still 100 (her VM, her state)
    r_alice = await sm.execute("alice", {"type": "python", "code": "print(f'Alice: secret={secret!r}  counter={counter}')"})

    # Bob's counter is still 999 (completely separate VM)
    r_bob = await sm.execute("bob", {"type": "python", "code": "print(f'Bob:   secret={secret!r}  counter={counter}')"})

    return r_alice, r_bob

r_alice, r_bob = await run_sessions()

print("── Session isolation demo ──")
print(r_alice["output"].strip())
print(r_bob["output"].strip())

sessions = sm.list_sessions()
print(f"\n── Active sessions ({len(sessions)}) ──")
for s in sessions:
    print(f"  session={s['session_id']:8s}  vm={s['vm_id']}  execs={s['exec_count']}  idle={s['idle_seconds']}s")


── Session isolation demo ──
Alice: secret='alice_data'  counter=100
Bob:   secret='bob_data'  counter=999

── Active sessions (2) ──
  session=alice     vm=96e546776ba1  execs=10  idle=0s
  session=bob       vm=68c6d275fc0f  execs=2  idle=0s


## 6. CodeInterpreterTool — BaseTool Integration

The `CodeInterpreterTool` wraps `SessionManager` as a proper `BaseTool` that agents can use.  
`tool.session_id` is set by the framework from the conversation's `thread_id`.

In [9]:
from agent_framework.tools.code_interpreter import CodeInterpreterTool

# Share the existing SessionManager (reuse warm pool + sessions already running)
tool = CodeInterpreterTool(config=config, session_manager=sm)

# Use session "alice" which already has a running VM — no new boot needed
tool.session_id = "alice"

# Call 1 — Python (stateful — phi gets defined)
r1 = await tool.execute(code="import math; phi = (1 + 5**0.5) / 2; print(f'φ = {phi:.6f}')")
print("── Tool call 1 (python) ──")
import json; d = json.loads(r1.content[0]["text"])
print(d.get("output", d.get("error")))

# Call 2 — phi still defined in alice's session
r2 = await tool.execute(code="print(f'π × φ = {3.14159 * phi:.6f}')")
print("\n── Tool call 2 (phi persists) ──")
print(json.loads(r2.content[0]["text"])["output"])

# Call 3 — bash mode
r3 = await tool.execute(code="df -h / && echo '---' && free -m", exec_type="bash")
print("\n── Tool call 3 (bash) ──")
print(json.loads(r3.content[0]["text"])["output"])

# Show the tool's JSON schema (what the LLM sees)
print("\n── LLM input_schema ──")
print(json.dumps(tool.input_schema, indent=2))


── Tool call 1 (python) ──
φ = 1.618034

── Tool call 2 (phi persists) ──
π × φ = 5.083199

── Tool call 3 (bash) ──
Filesystem      Size  Used Avail Use% Mounted on
/dev/root       2.0G  858M 1006M  47% /
---
               total        used        free      shared  buff/cache   available
Mem:             230          74          68           1          97         156
Swap:              0           0           0

── LLM input_schema ──
{
  "type": "object",
  "properties": {
    "code": {
      "type": "string",
      "description": "Python code to execute (when exec_type='python') or a bash command string (when exec_type='bash'). Use print() to show output."
    },
    "exec_type": {
      "type": "string",
      "enum": [
        "python",
        "bash"
      ],
      "description": "'python' (default) \u2014 exec in persistent namespace; 'bash' \u2014 run in /bin/bash subprocess",
      "default": "python"
    },
    "timeout": {
      "type": "integer",
      "description": "Max 

## 7. Cleanup

In [10]:
await sm.stop()
print("✅ All session VMs destroyed, warm pool stopped")


✅ All session VMs destroyed, warm pool stopped
